# HIV Model
Sam Daitzman / Adi Sudhakar

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [3]:
state = State(
    R=1000, # activated normal lymphocytes
    L=0, # latently infected
    E=0, # actively infected
    V=100 # free virions
)

,values
R,1000
L,0
E,0
V,100


In [9]:
system = System(
    gamma=1.36, # birthrate of R
    mu=1.36e-3, # natural deathrate of R and L
    delta=0.33, # deathrate of E
    beta=0.00027, # R-to-L conversion rate (per virion)
    alpha=3.6e-2, # L-to-E rate
    pie=100, # virion creation rate
    sigma=2, # virion death rate
    rho=0.1 # proportion of cells becoming latently infected upon infection
)

,values
gamma,0.00000
mu,0.00136
delta,0.00000
beta,0.00027
alpha,0.00000
pie,0.00000
sigma,0.00000
rho,0.00000


0.00136